In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import pandas as pd
import geopandas as gpd
from keplergl import KeplerGl
from tqdm.auto import tqdm
from src.settings import *
from src.tools.osmnx_utils import get_place_dir_name
from src.tools.vis_utils import visualize_kepler, save_config
from src.tools.feature_extraction import FEATURESET
tqdm.pandas()

In [8]:
cities = pd.read_csv(RAW_DATA_DIR / "cities.csv")
# cities = cities[(cities.country == "Poland") & (cities.kacper)]
cities

,city,country,kacper,szymon,piotr,kamil,regions
0,Tirana,Albania,False,True,False,False,NaN
1,Yerevan,Armenia,False,True,False,False,NaN
2,Sydney,Australia,True,False,False,False,NaN
3,Vienna,Austria,False,True,True,True,NaN
4,Baku,Azerbaijan,False,True,False,False,NaN
...,...,...,...,...,...,...,...
111,Los Angeles,United States,True,True,False,False,NaN
112,New York City,United States,True,True,False,False,NaN
113,Philadelphia,United States,False,True,False,False,NaN
114,San Francisco,United States,True,True,False,False,NaN


In [9]:
network_type = "drive"
features = list(FEATURESET.keys())

pbar = tqdm(cities.itertuples(), total=cities.shape[0])
edges = []
for row in pbar:
    place_name = f"{row.city},{row.country}"
    place_dir_name = get_place_dir_name(place_name)
    pbar.set_description(place_name)

    try:
        edges_city = gpd.read_file(GENERATED_DATA_DIR / place_dir_name / f"graph_{network_type}.gpkg", layer="edges")
        cols_to_add = set(features) - set(edges_city.columns)
        edges_city[list(cols_to_add)] = None
        edges_city["city"] = row.city
        edges_city["country"] = row.country
        edges.append(edges_city[features])
    except Exception as e:
        print("\nFailed", place_name, e)
    

edges = pd.concat(edges, ignore_index=True)

Budapest,Hungary:  35%|███▌      | 41/116 [00:42<01:10,  1.06it/s]


Failed Athens,Greece D:\Projekty\Programowanie\Studia\Semestr10\gis\osm-road-infrastructure\data\generated\Athens_Greece\graph_drive.gpkg: No such file or directory


London,United Kingdom:  92%|█████████▏| 107/116 [02:00<00:34,  3.88s/it] 


Failed Cardiff,United Kingdom D:\Projekty\Programowanie\Studia\Semestr10\gis\osm-road-infrastructure\data\generated\Cardiff_United-Kingdom\graph_drive.gpkg: No such file or directory


Washington D.C.,United States: 100%|██████████| 116/116 [02:25<00:00,  1.26s/it]


In [10]:
from itertools import chain

for f in features:
    feature_values_dirty = list(edges[f].value_counts().keys())
    feature_values_dirty = list(map(lambda x: eval(x) if "[" in str(x) else [str(x)], feature_values_dirty))
    feature_values_dirty = list(set(map(lambda x: str(x), chain(*feature_values_dirty))))
    feature_values_dirty.sort()
    print(f+":", feature_values_dirty, "\n")

oneway: ['False', 'True'] 

lanes: ['', '0', '1', '1.3', '1.5', '1.8', '10', '11', '12', '14', '15', '2', '20', '25', '2; 4', '2;4', '3', '30', '3c', '4', '4;2;4', '5', '6', '60', '7', '8', '9', 'no', 'way 403844716'] 

highway: ['P', 'bus', 'bus_stop', 'busway', 'closed', 'crossing', 'disused', 'emergency_bay', 'escape', 'living_street', 'motorway', 'motorway_link', 'no', 'passing_place', 'pedestrain', 'primary', 'primary_link', 'razed', 'residential', 'rest_area', 'road', 'secondary', 'secondary_link', 'stop', 'street_lamp', 'tertiary', 'tertiary_link', 'toll_gantry', 'trunk', 'trunk_link', 'unclassified', 'via_ferrata'] 

maxspeed: ['', '1', '10', '10 mph', '100', '100km/h', '110', '12 mph', '120', '120|110|100|90|80|80', '130', '140', '15', '15 mph', '160', '18 mph', '2', '2 mph', '20', '20 mph', '20,30,40', '200', '20;60', '21', '24.14', '25', '25 mph', '29', '3', '30', '30 mph', '30,40,50', '32', '35', '35 mph', '4 mph', '40', '40 mph', '403', '45', '45 mph', '5', '5 mph', '50', 